# 82841986 is_char and is_digit

# 82075350 regrex non-ascii and none-digit

## 86460763 left

In [ ]:
import os
import random
import re
import pandas as pd

: 

In [ ]:
max_length = 12
min_length = 3
root = '../data'
charset = 'abcdefghijklmnopqrstuvwxyz'
digits = '0123456789'

: 

In [3]:
def is_char(text, ratio=0.5):
    text = text.lower()
    length = max(len(text), 1)
    char_num = sum([t in charset for t in text])
    if char_num < min_length: return False
    if char_num / length < ratio: return False
    return True

def is_digit(text, ratio=0.5):
    length = max(len(text), 1)
    digit_num = sum([t in digits for t in text])
    if digit_num / length < ratio: return False
    return True

# generate training dataset

In [4]:
with open('/media/avlab/DATA/ABINet/workdir/wikitext-103/wiki.train.tokens', 'r') as file:
    lines = file.readlines()

In [5]:
inp, gt = [], []
for line in lines:
    token = line.lower().split()
    for text in token:
        text = re.sub('[^0-9a-zA-Z]+', '', text)
        if len(text) < min_length:
            # print('short-text', text)
            continue
        if len(text) > max_length:
            # print('long-text', text)
            continue
        inp.append(text)
        gt.append(text)

In [6]:
train_voc = os.path.join(root, 'WikiText-103.csv')
pd.DataFrame({'inp':inp, 'gt':gt}).to_csv(train_voc, index=None, sep='\t')

In [7]:
len(inp)

86460763

In [8]:
inp[:100]

['valkyria',
 'chronicles',
 'iii',
 'senj',
 'no',
 'valkyria',
 '3',
 'unk',
 'chronicles',
 'japanese',
 '3',
 'lit',
 'valkyria',
 'of',
 'the',
 'battlefield',
 '3',
 'commonly',
 'referred',
 'to',
 'as',
 'valkyria',
 'chronicles',
 'iii',
 'outside',
 'japan',
 'is',
 'a',
 'tactical',
 'role',
 'playing',
 'video',
 'game',
 'developed',
 'by',
 'sega',
 'and',
 'mediavision',
 'for',
 'the',
 'playstation',
 'portable',
 'released',
 'in',
 'january',
 '2011',
 'in',
 'japan',
 'it',
 'is',
 'the',
 'third',
 'game',
 'in',
 'the',
 'valkyria',
 'series',
 'employing',
 'the',
 'same',
 'fusion',
 'of',
 'tactical',
 'and',
 'real',
 'time',
 'gameplay',
 'as',
 'its',
 'predecessors',
 'the',
 'story',
 'runs',
 'parallel',
 'to',
 'the',
 'first',
 'game',
 'and',
 'follows',
 'the',
 'nameless',
 'a',
 'penal',
 'military',
 'unit',
 'serving',
 'the',
 'nation',
 'of',
 'gallia',
 'during',
 'the',
 'second',
 'europan',
 'war',
 'who',
 'perform',
 'secret',
 'black']

# generate evaluation dataset

In [9]:
def disturb(word, degree, p=0.3):
    if len(word) // 2 < degree: return word
    if is_digit(word): return word
    if random.random() < p: return word
    else:
        index = list(range(len(word)))
        random.shuffle(index)
        index = index[:degree]
        new_word = []
        for i in range(len(word)):
            if i not in index: 
                new_word.append(word[i])
                continue
            if (word[i] not in charset) and (word[i] not in digits):
                # special token
                new_word.append(word[i])
                continue
            op = random.random()
            if op < 0.1: # add
                new_word.append(random.choice(charset))
                new_word.append(word[i])
            elif op < 0.2: continue  # remove
            else: new_word.append(random.choice(charset))  # replace
        return ''.join(new_word)

In [10]:
lines = inp
degree = 1
keep_num = 50000

random.shuffle(lines)
part_lines = lines[:keep_num]
inp, gt = [], []

for w in part_lines:
    w = w.strip().lower()
    new_w = disturb(w, degree)
    inp.append(new_w)
    gt.append(w)
    
eval_voc = os.path.join(root, f'ReCTS-CTE_eval_d{degree}.csv')
pd.DataFrame({'inp':inp, 'gt':gt}).to_csv(eval_voc, index=None, sep='\t')

In [ ]:
list(zip(inp, gt))[:100]